## Training :

In [ ]:
import train
import torch
import utils

use_svhn = False
valid_size = 0.5
valid_shuffle = True
batch_size = 32
epochs = 10
learning_rate = 0.01
OCL_sw = True
alpha = 0.5
seed = None
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_interval = 100
model_save_path = utils.get_model_save_path(use_svhn, OCL_sw)


train.main(
    use_svhn=use_svhn,
    valid_size=valid_size,
    valid_shuffle=valid_shuffle,
    batch_size=batch_size,
    epochs=epochs,
    learning_rate=learning_rate,
    OCL_sw=OCL_sw,
    alpha=alpha,
    seed=seed,
    device=device,
    model_save_path=model_save_path,
    save_interval=save_interval
    )

## Evaluation :

In [ ]:
import torch
import test
import utils
import pandas as pd
use_svhn_options = [True, False]
OCL_sw_options = [True, False]
overlap_options = [0, 0.25, 0.5, 0.75, 1]
batch_size_test = 1024
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
results_dict = {
    'use_svhn': [],
    'OCL_sw': [],
    'test_accuracy': []
}

for use_svhn in use_svhn_options:
    test_set = utils.initialize_dataset('test', use_svhn)
    loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test)
    for OCL_sw in OCL_sw_options:
        network = utils.initialize_network(use_svhn, OCL_sw, device=device)
        model_save_path = utils.get_model_save_path(use_svhn, OCL_sw)
        network = utils.load_checkpoint_to_model(network, model_save_path, device=device)


        test_accuracy = test.evaluate_model(network, loader_test, device)
        results_dict['use_svhn'].append(use_svhn)
        results_dict['OCL_sw'].append(OCL_sw)
        results_dict['test_accuracy'].append(test_accuracy)

results_df = pd.DataFrame(results_dict)
results_df.to_csv('test_results.csv', index=False)




In [ ]:

results_dict = {
    'use_svhn': [],
    'OCL_sw': [],
    'overlap': [],
    'test_accuracy': []
}


for use_svhn in use_svhn_options:
    test_set = utils.initialize_dataset('test', use_svhn)
    loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test)
    for OCL_sw in OCL_sw_options:
        network = utils.initialize_network(use_svhn, OCL_sw, device=device)
        model_save_path = utils.get_model_save_path(use_svhn, OCL_sw)
        network = utils.load_checkpoint_to_model(network, model_save_path, device=device)
        for overlap in overlap_options:
            test_accuracy = test.evaluate_model_with_overlap(network, loader_test, device, overlap)
            results_dict['use_svhn'].append(use_svhn)
            results_dict['OCL_sw'].append(OCL_sw)
            results_dict['overlap'].append(overlap)
            results_dict['test_accuracy'].append(test_accuracy)

results_df = pd.DataFrame(results_dict)
results_df.to_csv('test_results_overlap.csv', index=False)

## Visualization :

In [ ]:
import torch
import test
import utils
import pandas as pd
import vis

def plot_representations(use_svhn, ocl_sw, n_samples, device, batch_size_test=1024, step=None):
    models_save_dir = './models/'
    fig_save_dir = './figures/'
    dataset_name = 'SVHN' if use_svhn else 'MNIST'
    model_tag = 'OCL' if OCL_sw else 'NoOCL'
    base_name = f'{dataset_name}_{model_tag}'
    if step is None:
        model_save_path = f'{models_save_dir}{base_name}.pt'
        fig_save_path = f'{fig_save_dir}{base_name}.png'
        title = f'{dataset_name} {model_tag}'
    else:
        model_save_path = f'{models_save_dir}{base_name}_{step}.pt'
        fig_save_path = f'{fig_save_dir}{base_name}_{step}.png'
        title = f'VGG16 with {model_tag} @ {step}'
        
    test_set = utils.initialize_dataset('test', use_svhn)
    loader_test = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test)
    network = utils.initialize_network(use_svhn, ocl_sw, device=device)
    network = utils.load_checkpoint_to_model(network, model_save_path, device=device)
    representations, labels = vis.get_representations(network, loader_test, device=device, num_samples=n_samples)
    vis.plot_representations_tsne_2d(representations, labels, title=title, show=True, save_path=fig_save_path)


In [ ]:
OCL_sw = True
use_svhn = False
batch_size_test = 128
n_samples = 6000
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
steps = list(range(100, 3300, 100))
for step in steps:
    plot_representations(use_svhn, OCL_sw, n_samples, device, batch_size_test=batch_size_test, step=step)

In [ ]:
# create gif from images
import imageio
import os
import glob

def create_gif(gif_name, path, duration=0.5):
    frames = []
    png_files = glob.glob(os.path.join(path, '*.png'))
    png_files.sort(key=os.path.getmtime)
    for file_name in png_files:
        frames.append(imageio.imread(file_name))
    imageio.mimsave(gif_name, frames, 'GIF', duration=duration)

gif_name = 'MNIST_OCL.gif'
path = './figures/'
create_gif(gif_name, path, duration=350)